### load data

In [1]:
from pandas import read_pickle
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\full_train2.pkl"
data = read_pickle(path)
X_cols = [col for col in data.columns if col not in ["card_id","target"]]
X, y = data[X_cols], data['target']

In [2]:
categorical_feature = [
    'feature1_1', 'feature1_2', 'feature1_3', 'feature1_4', 'feature1_5', 'feature2_1', 'feature2_2', 'feature2_3', 'feature3_0', 'feature3_1',
    "feature_sum","feature_max","feature_min", "quarter","feature_1","feature_2","feature_3",'first_active_month_month',
    'first_active_month_year', 'old_purchase_date_max_month',
    'old_purchase_date_max_year', 'old_purchase_date_min_month',
    'old_purchase_date_min_year','new_purchase_date_max_month', 'new_purchase_date_max_year',
    'new_purchase_date_min_month', 'new_purchase_date_min_year'
] 
categorical_feature = [i for i,col in enumerate(X.columns) if col in categorical_feature]

### commence optuna

In [4]:
from catboost import Pool, cv
from optuna.integration import CatBoostPruningCallback
from optuna import create_study

In [6]:
cv_dataset = Pool(data=X,
                  label=y,
                  cat_features=None)

In [20]:
def objective(trial):

    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01,0.03),
        "l2_leaf_reg": 0.0038237851526781623, #trial.suggest_float("l2_leaf_reg", 0.001,0.01),
        "subsample": 0.7305267944941545, # trial.suggest_float("subsample", 0.7,0.9),
        "rsm": 0.725781808210217, # trial.suggest_float('rsm', 0.65, 0.85),
        "depth": 7 ,# trial.suggest_int('depth', 4, 8),
        "grow_policy": "SymmetricTree", # trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise"]),
        "min_data_in_leaf": 404, # trial.suggest_int('min_data_in_leaf', 400, 600),
        "nan_mode": "Min", # trial.suggest_categorical("nan_mode", ["Min", "Max"]),
        "loss_function":"RMSE",
        "eval_metric": "RMSE",
        "use_best_model": True,
        "random_seed": 0, 
        "verbose": True,
        "early_stopping_rounds": 10
    }
    
    pruning_callback = CatBoostPruningCallback(trial, "RMSE")
    
    eval_hist = cv(
        pool=cv_dataset,
        params=params,
        iterations=700,
        fold_count=2,
        seed=1,
        shuffle=True,
        verbose_eval=True,
        early_stopping_rounds=10,
        return_models=False# ,
        # callbacks  = [pruning_callback]
    )   
    return eval_hist["test-RMSE-mean"].min()

In [21]:
study = create_study(
    direction='minimize',                     
    sampler= None,          # use TPESampler if None
    pruner= None         # use median Sampler if None
)
study.optimize(objective, n_trials=10)

[I 2022-10-19 14:26:40,188] A new study created in memory with name: no-name-63345f08-9251-451c-a79b-c75a3ba2535d
C:\Users\tanch\AppData\Local\Temp\ipykernel_10676\3663714428.py:21: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")


Training on fold [0/2]
0:	learn: 3.8601749	test: 3.8726247	best: 3.8726247 (0)	total: 117ms	remaining: 1m 21s
1:	learn: 3.8556378	test: 3.8682711	best: 3.8682711 (1)	total: 249ms	remaining: 1m 27s
2:	learn: 3.8515601	test: 3.8642356	best: 3.8642356 (2)	total: 407ms	remaining: 1m 34s
3:	learn: 3.8473272	test: 3.8599245	best: 3.8599245 (3)	total: 551ms	remaining: 1m 35s
4:	learn: 3.8431677	test: 3.8559501	best: 3.8559501 (4)	total: 696ms	remaining: 1m 36s
5:	learn: 3.8396651	test: 3.8524518	best: 3.8524518 (5)	total: 835ms	remaining: 1m 36s
6:	learn: 3.8359713	test: 3.8490885	best: 3.8490885 (6)	total: 1.01s	remaining: 1m 40s
7:	learn: 3.8323860	test: 3.8455794	best: 3.8455794 (7)	total: 1.16s	remaining: 1m 40s
8:	learn: 3.8290351	test: 3.8420994	best: 3.8420994 (8)	total: 1.31s	remaining: 1m 40s
9:	learn: 3.8256687	test: 3.8389024	best: 3.8389024 (9)	total: 1.51s	remaining: 1m 44s
10:	learn: 3.8220590	test: 3.8354584	best: 3.8354584 (10)	total: 1.68s	remaining: 1m 44s
11:	learn: 3.81861

94:	learn: 3.6853723	test: 3.7185136	best: 3.7185136 (94)	total: 13.5s	remaining: 1m 25s
95:	learn: 3.6848008	test: 3.7181291	best: 3.7181291 (95)	total: 13.6s	remaining: 1m 25s
96:	learn: 3.6840646	test: 3.7176902	best: 3.7176902 (96)	total: 13.7s	remaining: 1m 25s
97:	learn: 3.6833143	test: 3.7172802	best: 3.7172802 (97)	total: 13.9s	remaining: 1m 25s
98:	learn: 3.6826147	test: 3.7168123	best: 3.7168123 (98)	total: 14s	remaining: 1m 25s
99:	learn: 3.6820121	test: 3.7165645	best: 3.7165645 (99)	total: 14.1s	remaining: 1m 24s
100:	learn: 3.6813231	test: 3.7161533	best: 3.7161533 (100)	total: 14.3s	remaining: 1m 24s
101:	learn: 3.6805713	test: 3.7156543	best: 3.7156543 (101)	total: 14.4s	remaining: 1m 24s
102:	learn: 3.6799798	test: 3.7154486	best: 3.7154486 (102)	total: 14.5s	remaining: 1m 24s
103:	learn: 3.6794530	test: 3.7151260	best: 3.7151260 (103)	total: 14.7s	remaining: 1m 24s
104:	learn: 3.6786562	test: 3.7147485	best: 3.7147485 (104)	total: 14.8s	remaining: 1m 23s
105:	learn: 3

186:	learn: 3.6348983	test: 3.6969086	best: 3.6969086 (186)	total: 25.8s	remaining: 1m 10s
187:	learn: 3.6343947	test: 3.6967892	best: 3.6967892 (187)	total: 25.9s	remaining: 1m 10s
188:	learn: 3.6341025	test: 3.6967649	best: 3.6967649 (188)	total: 26.1s	remaining: 1m 10s
189:	learn: 3.6337478	test: 3.6966833	best: 3.6966833 (189)	total: 26.2s	remaining: 1m 10s
190:	learn: 3.6335107	test: 3.6966178	best: 3.6966178 (190)	total: 26.3s	remaining: 1m 10s
191:	learn: 3.6331302	test: 3.6965036	best: 3.6965036 (191)	total: 26.5s	remaining: 1m 9s
192:	learn: 3.6326845	test: 3.6964140	best: 3.6964140 (192)	total: 26.6s	remaining: 1m 9s
193:	learn: 3.6323029	test: 3.6963552	best: 3.6963552 (193)	total: 26.7s	remaining: 1m 9s
194:	learn: 3.6319369	test: 3.6962402	best: 3.6962402 (194)	total: 26.8s	remaining: 1m 9s
195:	learn: 3.6314030	test: 3.6961680	best: 3.6961680 (195)	total: 27s	remaining: 1m 9s
196:	learn: 3.6308830	test: 3.6960460	best: 3.6960460 (196)	total: 27.1s	remaining: 1m 9s
197:	le

278:	learn: 3.5994325	test: 3.6890083	best: 3.6890083 (278)	total: 37.6s	remaining: 56.7s
279:	learn: 3.5989714	test: 3.6889437	best: 3.6889437 (279)	total: 37.7s	remaining: 56.6s
280:	learn: 3.5986320	test: 3.6888152	best: 3.6888152 (280)	total: 37.9s	remaining: 56.5s
281:	learn: 3.5983505	test: 3.6887689	best: 3.6887689 (281)	total: 38s	remaining: 56.3s
282:	learn: 3.5980407	test: 3.6887214	best: 3.6887214 (282)	total: 38.1s	remaining: 56.1s
283:	learn: 3.5975848	test: 3.6886324	best: 3.6886324 (283)	total: 38.2s	remaining: 56s
284:	learn: 3.5971820	test: 3.6885283	best: 3.6885283 (284)	total: 38.4s	remaining: 55.9s
285:	learn: 3.5969504	test: 3.6884910	best: 3.6884910 (285)	total: 38.5s	remaining: 55.7s
286:	learn: 3.5965603	test: 3.6884245	best: 3.6884245 (286)	total: 38.6s	remaining: 55.6s
287:	learn: 3.5962443	test: 3.6883893	best: 3.6883893 (287)	total: 38.7s	remaining: 55.4s
288:	learn: 3.5957452	test: 3.6882795	best: 3.6882795 (288)	total: 38.9s	remaining: 55.3s
289:	learn: 3.

...

483:	learn: 3.5415879	test: 3.6847328	best: 3.6847328 (483)	total: 1m 9s	remaining: 30.8s
484:	learn: 3.5412555	test: 3.6847293	best: 3.6847293 (484)	total: 1m 9s	remaining: 30.7s
485:	learn: 3.5409918	test: 3.6847090	best: 3.6847090 (485)	total: 1m 9s	remaining: 30.5s
486:	learn: 3.5406509	test: 3.6845977	best: 3.6845977 (486)	total: 1m 9s	remaining: 30.4s
487:	learn: 3.5403950	test: 3.6845868	best: 3.6845868 (487)	total: 1m 9s	remaining: 30.2s
488:	learn: 3.5402003	test: 3.6845746	best: 3.6845746 (488)	total: 1m 9s	remaining: 30.1s
489:	learn: 3.5398816	test: 3.6844954	best: 3.6844954 (489)	total: 1m 9s	remaining: 29.9s
490:	learn: 3.5397442	test: 3.6844850	best: 3.6844850 (490)	total: 1m 10s	remaining: 29.8s
491:	learn: 3.5395252	test: 3.6844468	best: 3.6844468 (491)	total: 1m 10s	remaining: 29.6s
492:	learn: 3.5392680	test: 3.6844287	best: 3.6844287 (492)	total: 1m 10s	remaining: 29.5s
493:	learn: 3.5390552	test: 3.6844096	best: 3.6844096 (493)	total: 1m 10s	remaining: 29.4s
494:	l

575:	learn: 3.5183186	test: 3.6829122	best: 3.6828924 (573)	total: 1m 22s	remaining: 17.7s
576:	learn: 3.5180896	test: 3.6828988	best: 3.6828924 (573)	total: 1m 22s	remaining: 17.5s
577:	learn: 3.5178626	test: 3.6828814	best: 3.6828814 (577)	total: 1m 22s	remaining: 17.4s
578:	learn: 3.5175107	test: 3.6829223	best: 3.6828814 (577)	total: 1m 22s	remaining: 17.2s
579:	learn: 3.5173192	test: 3.6829119	best: 3.6828814 (577)	total: 1m 22s	remaining: 17.1s
580:	learn: 3.5170516	test: 3.6829077	best: 3.6828814 (577)	total: 1m 22s	remaining: 16.9s
581:	learn: 3.5168457	test: 3.6828825	best: 3.6828814 (577)	total: 1m 22s	remaining: 16.8s
582:	learn: 3.5165726	test: 3.6828810	best: 3.6828810 (582)	total: 1m 22s	remaining: 16.7s
583:	learn: 3.5163567	test: 3.6828752	best: 3.6828752 (583)	total: 1m 23s	remaining: 16.5s
584:	learn: 3.5162117	test: 3.6828785	best: 3.6828752 (583)	total: 1m 23s	remaining: 16.4s
585:	learn: 3.5159064	test: 3.6828896	best: 3.6828752 (583)	total: 1m 23s	remaining: 16.2s

667:	learn: 3.4957110	test: 3.6818618	best: 3.6818618 (667)	total: 1m 35s	remaining: 4.56s
668:	learn: 3.4955640	test: 3.6818241	best: 3.6818241 (668)	total: 1m 35s	remaining: 4.42s
669:	learn: 3.4952851	test: 3.6818169	best: 3.6818169 (669)	total: 1m 35s	remaining: 4.27s
670:	learn: 3.4950703	test: 3.6817979	best: 3.6817979 (670)	total: 1m 35s	remaining: 4.13s
671:	learn: 3.4947505	test: 3.6817625	best: 3.6817625 (671)	total: 1m 35s	remaining: 3.99s
672:	learn: 3.4945063	test: 3.6817492	best: 3.6817492 (672)	total: 1m 35s	remaining: 3.85s
673:	learn: 3.4942393	test: 3.6816580	best: 3.6816580 (673)	total: 1m 36s	remaining: 3.7s
674:	learn: 3.4940459	test: 3.6816673	best: 3.6816580 (673)	total: 1m 36s	remaining: 3.56s
675:	learn: 3.4937929	test: 3.6816580	best: 3.6816580 (673)	total: 1m 36s	remaining: 3.42s
676:	learn: 3.4935173	test: 3.6816638	best: 3.6816580 (673)	total: 1m 36s	remaining: 3.28s
677:	learn: 3.4933644	test: 3.6816655	best: 3.6816580 (673)	total: 1m 36s	remaining: 3.13s


[I 2022-10-19 14:53:01,733] Trial 9 finished with value: 3.679119890268486 and parameters: {'learning_rate': 0.014546117782899482}. Best is trial 0 with value: 3.6781087244185757.


690:	learn: 3.4902031	test: 3.6816419	best: 3.6816023 (681)	total: 1m 38s	remaining: 1.28s
691:	learn: 3.4899247	test: 3.6816206	best: 3.6816023 (681)	total: 1m 38s	remaining: 1.14s

bestTest = 3.681602267
bestIteration = 681



In [17]:
study.best_trial.params, study.best_trial.value

({'learning_rate': 0.02861036040317381, 'l2_leaf_reg': 0.0038237851526781623},
 3.6742563012813534)